# PROJET MACH_BDA_DATAVIZ

In [ ]:
import pandas as pd
from datasets import load_dataset
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
dataset = load_dataset("d0r1h/customer_churn")

df = pd.DataFrame(dataset['train'])

df_reset = df # copie pour ne pas recharger tout le dataset

In [ ]:
def reset_dataframe():
    return df_reset

In [ ]:
df.iloc[0]

### Traitement des Données

In [ ]:
df.isnull().sum()

In [ ]:
def clean_dataframe(df):
    df.dropna(inplace=True)
    df = df[df['days_since_last_login'] > 0]
    df = df[df['avg_time_spent'] > 0]
    df = df[df['points_in_wallet'] > 0]
    return df

In [ ]:
df = clean_dataframe(df)

In [ ]:
df.describe()

### Analyse et visualisation exploratoire des Données

### Evaluation de plusieurs modèles avec les bonnes métriques

Standardisation des données numériques (mettre toutes les valeurs entre 0 et 1, pas sûr s'il y a vraiment besoin):

In [ ]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

df[numeric_columns] = MinMaxScaler().fit_transform(df[numeric_columns])

## Toute la suite est juste un test simple pour savoir ce qu'il me reste a clean dans le dataset -- PAUL --

In [ ]:
df = df.drop(columns=df.select_dtypes(exclude=['float64', 'int64']).columns)

In [ ]:
class PrevisionChurn:
    def __init__(self, model):
        self.model = model
    
    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)
    
    def predict(self, X_test):
        return self.model.predict(X_test)
    
    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return accuracy_score(y_test, y_pred)

In [ ]:
X = df.drop(columns=['churn_risk_score'])
y = df['churn_risk_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()

predictive_model = PrevisionChurn(model)

predictive_model.train(X_train, y_train)

accuracy = predictive_model.evaluate(X_test, y_test)
print("Accuracy:", np.round(accuracy * 100,2))


### Visualiser et interpreter les résultats